# Review Instructions

Please review the MSv4 `field_and_source_xds` schema and the XRADIO interface (`ps['MSv4_name'].VISIBILITY.field_and_source_xds`). Note that the PS (processing set) interface or the main_xds should not be reviewed.

The `field_and_source_xds` schema specification: https://docs.google.com/spreadsheets/d/14a6qMap9M5r_vjpLnaBKxsR9TF4azN5LVdOxLacOX-s/edit#gid=1658760192

## Preparatory Material
Go over Xarray nomenclature and selection syntax:
- https://docs.xarray.dev/en/latest/user-guide/terminology.html
- https://docs.xarray.dev/en/latest/user-guide/indexing.html

MSv2 and CASA documentation:
- MSv2 schema: https://casacore.github.io/casacore-notes/229.pdf
- MSv3 schema: https://casacore.github.io/casacore-notes/264.pdf
- Ephemeris Data in CASA: https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data

## `field_and_source_xds` Schema
The FIELD, SOURCE, and EPHEMERIS tables in the MSv2 contain closely related information:
- **FIELD**: Field position for a source.
- **SOURCE**: Information about the source being observed (position, proper motion, etc.)
- **EPHEMERIS**: Ephemerides of the source.

These can be combined into a single dataset for MSv4 because it consists of a single field and consequently a single source[^1].

### Use Cases
The use cases considered during the design of the schema were:
- Single field observation (type=standard).
- Mosaic observation (type=standard).
- Ephemeris observation (type=ephemeris).
- Mosaic Ephemeris observation (type=ephemeris).

To satisfy these use cases, two types of `field_and_source_xds` were created: standard and ephemeris. The main difference is that the ephemeris type has a `FIELD_PHASE_OFFSET` data variable that is relative to the `SOURCE_POSITION/SOURCE_DIRECTION` data variable (contains the ephemerides and has a time axis), while the standard type has `FIELD_PHASE/DELAY/REFERENCE_CENTERS` and `SOURCE_POSITION` (has no time axis). The `SOURCE_POSITION/DIRECTION` is kept separate from the `FIELD_PHASE_OFFSET/CENTER` so that the intent `OBSERVE_TARGET#OFF_SOURCE` is supported and the ephemeris can be easily changed.

## Key Questions to Answer
### Schema Questions
- 1.1) Are there missing use cases?
- 1.2) Is all the information present needed for offline processing?
  - 1.2.1) Are there data variables we need to add (for example, the JPL Horizons data has additional information such as the North pole position angle, etc., see [CASA Ephemeris Data](https://casadocs.readthedocs.io/en/latest/notebooks/external-data.html#Ephemeris-Data))?
  - 1.2.2) In some CASA ephemeris tables, there are table keywords such as `VS_CREATE`, `VS_DATE`, `VS_TYPE`, `VS_VERSION`, `MJD0`, `dMJD`, `earliest`, `latest`, `radii`, `meanradm`, `orb_per`, `rot_per`. Do we need any of these?
- 1.3) Is there a use case where the `FIELD_PHASE_CENTER` and `FIELD_DELAY_CENTER` would differ (i.e., do we need to store both)?
- 1.4) For interferometer observations, do we need to store the `FIELD_REFERENCE_CENTER` or can this be omitted (will still be present for Single dish)?
- 1.5) The ephemeris data is recorded in degrees, AU, and MJD. Should these be converted to radians, meters, and time (Unix)? Note that each data variable has measurement information attached to it. For example:
```Python
  frame: ICRS
  type: sky_coord
  units: ['deg', 'deg', 'AU']
```

- 1.6) For ephemeris observations, should we add the SOURCE_PROPER_MOTION if available?
- 1.7) Is the name `field_and_source_xds` sufficiently descriptive?
- 1.8) Should we also add the DOPPLER table information to the schema (if so, any idea where we can get an MSv2 with a DOPPLER table)?
- 1.9) Any naming suggestions or data layout?

### XRADIO
2.1) After reviewing the XARRAY documentation and the descriptions of the data variables in the `field_and_source_xds` schema, do you find the XARRAY interface intuitive and easy to use?



[^1]: This is inhereted from MSv2 that only allows a single source per field [https://casacore.github.io/casacore-notes/229.pdf, p35], though a source can appear in more than one field.


# Environment instructions

It is recommended to use the conda environment manager to create a clean, self-contained runtime where xradio and all its dependencies can be installed:

```bash
conda create --name xradio python=3.11 --no-default-packages
conda activate xradio
```

Clone the repository, checkout the review branch and do a local install:

```bash
git clone https://github.com/casangi/xradio.git
git checkout 162-create-combined-field-source-and-ephemeris-dataset
cd xradio
pip install -e .
```

# ALMA Example

An ephemeris mosaic observation of the sun.

ALMA archive file downloaded: https://almascience.nrao.edu/dataPortal/2022.A.00001.S_uid___A002_X1003af4_X75a3.asdm.sdm.tar 

- Project: 2022.A.00001.S
- Member ous id (MOUS): uid://A001/X3571/X130
- Group ous id (GOUS): uid://A001/X3571/X131

CASA commands used to create the dataset:
```python
importasdm(asdm='uid___A002_X1003af4_X75a3.asdm.sdm',vis='uid___A002_X1003af4_X75a3.ms',asis='Ephemeris Antenna Station Receiver Source CalAtmosphere CalWVR',bdfflags=True,with_pointing_correction=True,convert_ephem2geo=True)

importasdm(asdm='22A-347.sb41163051.eb41573599.59642.62832791667.asdm',vis='22A-347.sb41163051.eb41573599.59642.62832791667.ms',asis='Ephemeris Antenna Station Receiver Source',with_pointing_correction=True,convert_ephem2geo=True)

mstransform(vis='ALMA_uid___A002_X1003af4_X75a3.split.ms',outputvis='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms',createmms=False,timeaverage=True,timebin='2s',timespan='scan',reindex=True)

import numpy as np
tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::FLAG_CMD',nomodify=False)
tb.removerows(np.arange(tb.nrows())) # tb.removerows(np.arange(76401))
tb.flush()
tb.done()

tb.open('ALMA_uid___A002_X1003af4_X75a3.split.avg.ms::POINTING',nomodify=False)
tb.removerows(np.arange(tb.nrows()))   #tb.removerows(np.arange(5617115))
tb.flush()
tb.done()
```


## Download Data

In [2]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set
from xradio.vis.read_processing_set import read_processing_set
import graphviper

graphviper.utils.data.download(file="ALMA_uid___A002_X1003af4_X75a3.split.avg.ms")

 Download List                                
 ───────────────────────────────────────────── 
  ALMA_uid___A002_X1003af4_X75a3.split.avg.ms

ALMA_uid___A002_X1003af4_X75a3.split.avg.ms.zip:   0%|          | 0.00/47.8M [00:00<?, ?iB/s]

# Start Dask cluster 
Choose an approriate number of cores and memory_limit (this is per core).

In [3]:
from graphviper.dask.client import local_client

viper_client = local_client(cores=4, memory_limit="4GB")
viper_client

[2024-06-11 17:21:41,246]     INFO  graphviper:  Checking parameter values for client.local_client 
[2024-06-11 17:21:41,247]     INFO  graphviper:  Module path: /Users/jsteeb/Dropbox/graphviper/ 
[2024-06-11 17:21:41,247]     INFO  graphviper:  Searching /Users/jsteeb/Dropbox/graphviper/src/graphviper/config/ for configuration file, please wait ... 
[2024-06-11 17:21:41,253]  WARNING      client:  It is recommended that the local cache directory be set using the dask_local_dir parameter. 


/Users/jsteeb/Downloads/yes/envs/zinc/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 64384 instead
  warnings.warn(


[2024-06-11 17:21:42,250]     INFO      client:  Created client <MenrvaClient: 'tcp://127.0.0.1:64385' processes=4 threads=4, memory=14.90 GiB> 


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:64384/status,
Dashboard: http://127.0.0.1:64384/status,Workers: 4
Total threads: 4,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:64385,Workers: 4
Dashboard: http://127.0.0.1:64384/status,Total threads: 4
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:64399,Total threads: 1
Dashboard: http://127.0.0.1:64403/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:64388,


# Convert dataset

In [4]:
from xradio.vis.convert_msv2_to_processing_set import convert_msv2_to_processing_set

in_file='ALMA_uid___A002_X1003af4_X75a3.split.avg.ms'
out_file = "ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr"

convert_msv2_to_processing_set(
    in_file=in_file,
    out_file=out_file,
    partition_scheme="ddi_intent_field",
    parallel=True,
    overwrite=True,
)

# Inspect Processing Set

In [5]:
import pandas as pd

# Set the maximum number of rows displayed before scrolling
pd.set_option('display.max_rows', 1000)

from xradio.vis.read_processing_set import read_processing_set
ps = read_processing_set("ALMA_uid___A002_X1003af4_X75a3.split.avg.zarr")
ps.summary()

,name,ddi,intent,field_id,field_name,start_frequency,end_frequency,shape,field_coords
0,ALMA_uid___A002_X1003af4_X75a3.split.avg_310,0,OBSERVE_TARGET#ON_SOURCE,10,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
1,ALMA_uid___A002_X1003af4_X75a3.split.avg_328,0,OBSERVE_TARGET#ON_SOURCE,28,Sun_10,2.480000e+11,2.480000e+11,"(8, 51, 1, 2)",Ephemeris
2,ALMA_uid___A002_X1003af4_X75a3.split.avg_317,0,OBSERVE_TARGET#ON_SOURCE,17,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
3,ALMA_uid___A002_X1003af4_X75a3.split.avg_321,0,OBSERVE_TARGET#ON_SOURCE,21,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
4,ALMA_uid___A002_X1003af4_X75a3.split.avg_90,0,"CALIBRATE_ATMOSPHERE#HOT,CALIBRATE_WVR#HOT",0,Sun_10,2.480000e+11,2.480000e+11,"(3, 51, 1, 2)",Ephemeris
5,ALMA_uid___A002_X1003af4_X75a3.split.avg_319,0,OBSERVE_TARGET#ON_SOURCE,19,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
6,ALMA_uid___A002_X1003af4_X75a3.split.avg_326,0,OBSERVE_TARGET#ON_SOURCE,26,Sun_10,2.480000e+11,2.480000e+11,"(12, 51, 1, 2)",Ephemeris
7,ALMA_uid___A002_X1003af4_X75a3.split.avg_988,2,OBSERVE_TARGET#ON_SOURCE,28,Sun_10,2.299609e+11,2.300547e+11,"(6, 1326, 7, 2)",Ephemeris
8,ALMA_uid___A002_X1003af4_X75a3.split.avg_986,2,OBSERVE_TARGET#ON_SOURCE,26,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris
9,ALMA_uid___A002_X1003af4_X75a3.split.avg_972,2,OBSERVE_TARGET#ON_SOURCE,12,Sun_10,2.299609e+11,2.300547e+11,"(9, 1326, 7, 2)",Ephemeris


# Inspect field_and_source_xds: Standard Use case (non-ephemeris)

In [6]:
standard_field_and_source_xds = ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_902'].VISIBILITY.field_and_source_xds.load() # Load the data into memory
standard_field_and_source_xds

<xarray.Dataset> Size: 208B
Dimensions:                 (sky_dir_label: 2, line_name: 1)
Coordinates:
  * line_name               (line_name) <U22 88B 'Single_Continuum(ID=0)'
  * sky_dir_label           (sky_dir_label) <U3 24B 'ra' 'dec'
Data variables:
    FIELD_DELAY_CENTER      (sky_dir_label) float64 16B -2.579 -0.1374
    FIELD_PHASE_CENTER      (sky_dir_label) float64 16B -2.579 -0.1374
    FIELD_REFERENCE_CENTER  (sky_dir_label) float64 16B -2.579 -0.1374
    LINE_REST_FREQUENCY     (line_name) float64 8B 2.3e+11
    LINE_SYSTEMIC_VELOCITY  (line_name) float64 8B 0.0
    SOURCE_DIRECTION        (sky_dir_label) float64 16B -2.579 -0.1374
    SOURCE_PROPER_MOTION    (sky_dir_label) float64 16B 0.0 0.0
Attributes:
    code:                       none
    field_and_source_xds_type:  standard
    field_code:                 none
    field_id:                   2
    field_name:                 J1408-0752
    source_id:                  2
    source_name:                J1408-0752

In [7]:
standard_field_and_source_xds.attrs['field_id'] #How to access field_and_source_xds attributes.

2

In [8]:
standard_field_and_source_xds.FIELD_PHASE_CENTER #How to access field_and_source_xds datavariables. standard_field_and_source_xds['FIELD_PHASE_CENTER'] can also be used.

<xarray.DataArray 'FIELD_PHASE_CENTER' (sky_dir_label: 2)> Size: 16B
array([-2.57897986, -0.13742852])
Coordinates:
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['rad', 'rad']

In [9]:
standard_field_and_source_xds.FIELD_PHASE_CENTER.attrs #How to access field_and_source_xds datavariables measures information stored in the attributes.

{'frame': 'ICRS', 'type': 'sky_coord', 'units': ['rad', 'rad']}

# Inspect field_and_source_xds: Ephemeris Use case (Mosaic)



In [10]:
ephemeris_field_and_source_xds = ps['ALMA_uid___A002_X1003af4_X75a3.split.avg_310'].VISIBILITY.field_and_source_xds.load() # Load the data into memory
ephemeris_field_and_source_xds

<xarray.Dataset> Size: 2kB
Dimensions:                        (sky_dir_label: 2, spherical_pos_label: 3,
                                    time: 33, sky_pos_label: 3)
Coordinates:
  * sky_dir_label                  (sky_dir_label) <U3 24B 'ra' 'dec'
  * sky_pos_label                  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * spherical_pos_label            (spherical_pos_label) <U4 48B 'lon' ... 'd...
  * time                           (time) float64 264B 1.667e+18 ... 1.667e+18
Data variables:
    FIELD_DELAY_CENTER_OFFSET      (sky_dir_label) float64 16B 5.816e-05 -7.5...
    FIELD_PHASE_CENTER_OFFSET      (sky_dir_label) float64 16B 5.816e-05 -7.5...
    FIELD_REFERENCE_CENTER_OFFSET  (sky_dir_label) float64 16B 5.816e-05 -7.5...
    OBSERVATION_POSITION           (spherical_pos_label) float64 24B 0.0 0.0 0.0
    SOURCE_POSITION                (time, sky_pos_label) float64 792B 209.7 ....
    SOURCE_RADIAL_VELOCITY         (time) float64 264B -0.0002714 ... -0.0002727
Attributes:
    code:                       none
    field_and_source_xds_type:  ephemeris
    field_code:                 none
    field_id:                   10
    field_name:                 Sun_10
    source_id:                  0
    source_name:                Sun_10

In [11]:
ephemeris_field_and_source_xds.FIELD_DELAY_CENTER_OFFSET

<xarray.DataArray 'FIELD_DELAY_CENTER_OFFSET' (sky_dir_label: 2)> Size: 16B
array([ 5.81603684e-05, -7.55525348e-05])
Coordinates:
  * sky_dir_label  (sky_dir_label) <U3 24B 'ra' 'dec'
Attributes:
    frame:    ICRS
    type:     sky_coord_offset
    units:    ['rad', 'rad']

In [12]:
ephemeris_field_and_source_xds.SOURCE_POSITION

<xarray.DataArray 'SOURCE_POSITION' (time: 33, sky_pos_label: 3)> Size: 792B
array([[209.67570833, -12.06283333,   0.99443619],
       [209.68504167, -12.06619444,   0.99443113],
       [209.694375  , -12.06958333,   0.99442616],
       [209.70366667, -12.07294444,   0.99442128],
       [209.713     , -12.07633333,   0.9944165 ],
       [209.72233333, -12.07969444,   0.99441183],
       [209.73158333, -12.08305556,   0.99440728],
       [209.74091667, -12.08641667,   0.99440285],
       [209.75020833, -12.08977778,   0.99439855],
       [209.75945833, -12.09313889,   0.99439438],
       [209.76875   , -12.0965    ,   0.99439036],
       [209.778     , -12.09983333,   0.99438649],
       [209.78729167, -12.10319444,   0.99438277],
       [209.79654167, -12.10652778,   0.9943792 ],
       [209.80579167, -12.10988889,   0.99437579],
       [209.81504167, -12.11322222,   0.99437255],
       [209.82433333, -12.11655556,   0.99436946],
       [209.83358333, -12.11991667,   0.99436654],
       [209.84283333, -12.12325   ,   0.99436379],
       [209.852125  , -12.12658333,   0.9943612 ],
       [209.86133333, -12.12991667,   0.99435877],
       [209.870625  , -12.13319444,   0.99435651],
       [209.879875  , -12.13652778,   0.9943544 ],
       [209.889125  , -12.13986111,   0.99435245],
       [209.89841667, -12.14316667,   0.99435065],
       [209.90766667, -12.1465    ,   0.994349  ],
       [209.91695833, -12.14980556,   0.99434749],
       [209.92625   , -12.15313889,   0.99434611],
       [209.93554167, -12.15644444,   0.99434487],
       [209.94483333, -12.15972222,   0.99434375],
       [209.95416667, -12.16302778,   0.99434275],
       [209.96345833, -12.16633333,   0.99434186],
       [209.97275   , -12.16963889,   0.99434106]])
Coordinates:
  * sky_pos_label  (sky_pos_label) <U4 48B 'ra' 'dec' 'dist'
  * time           (time) float64 264B 1.667e+18 1.667e+18 ... 1.667e+18
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['deg', 'deg', 'AU']

In [13]:
ephemeris_field_and_source_xds.SOURCE_POSITION.sel(sky_pos_label='dec')

<xarray.DataArray 'SOURCE_POSITION' (time: 33)> Size: 264B
array([-12.06283333, -12.06619444, -12.06958333, -12.07294444,
       -12.07633333, -12.07969444, -12.08305556, -12.08641667,
       -12.08977778, -12.09313889, -12.0965    , -12.09983333,
       -12.10319444, -12.10652778, -12.10988889, -12.11322222,
       -12.11655556, -12.11991667, -12.12325   , -12.12658333,
       -12.12991667, -12.13319444, -12.13652778, -12.13986111,
       -12.14316667, -12.1465    , -12.14980556, -12.15313889,
       -12.15644444, -12.15972222, -12.16302778, -12.16633333,
       -12.16963889])
Coordinates:
    sky_pos_label  <U4 16B 'dec'
  * time           (time) float64 264B 1.667e+18 1.667e+18 ... 1.667e+18
Attributes:
    frame:    ICRS
    type:     sky_coord
    units:    ['deg', 'deg', 'AU']